# Diskcache Examples

Reference:

- [Quick Start](https://grantjenks.com/docs/diskcache/)
- [Tutorial](https://grantjenks.com/docs/diskcache/tutorial.html)

## Create Cache

In [1]:
import diskcache

In [5]:
import os
import time
from pathlib import Path

dir_here = Path(os.getcwd())
dir_cache = dir_here / ".cache"
cache = diskcache.Cache(str(dir_cache))

## Common Operation

- [__contains__, key in cache](https://grantjenks.com/docs/diskcache/api.html#diskcache.Cache.__contains__): Return True if key matching item is found in cache.
- [set](https://grantjenks.com/docs/diskcache/api.html#diskcache.Cache.set)(key, value, expire=None, read=False, tag=None, retry=False): Set key and value item in cache.
- [get](https://grantjenks.com/docs/diskcache/api.html#diskcache.Cache.get)(key, default=None, read=False, expire_time=False, tag=False, retry=False): Retrieve value from cache. If key is missing, return default.
- [delete](https://grantjenks.com/docs/diskcache/api.html#diskcache.Cache.delete)(key, retry=False): Delete corresponding item for key from cache.
- [add](https://grantjenks.com/docs/diskcache/api.html#diskcache.Cache.add)(key, value, expire=None, read=False, tag=None, retry=False):
- [pop](https://grantjenks.com/docs/diskcache/api.html#diskcache.Cache.pop)(key, default=None, expire_time=False, tag=False, retry=False): Remove corresponding item for key from cache and return value. If key is missing, return default.
- [touch](https://grantjenks.com/docs/diskcache/api.html#diskcache.Cache.touch)(key, expire=None, retry=False): Touch key in cache and update expire time.
- [clear](https://grantjenks.com/docs/diskcache/api.html#diskcache.Cache.clear)(retry=False): Remove all items from cache.
- [expire](https://grantjenks.com/docs/diskcache/api.html#diskcache.Cache.expire)(now=None, retry=False): Remove expired items from cache.
- [evict](https://grantjenks.com/docs/diskcache/api.html#diskcache.Cache.evict)(tag, retry=False): Remove items with matching tag from cache.
- [create_tag_index](https://grantjenks.com/docs/diskcache/api.html#diskcache.Cache.create_tag_index)(): Create tag index on cache database.
- [drop_tag_index](https://grantjenks.com/docs/diskcache/api.html#diskcache.Cache.drop_tag_index)(): Drop tag index on cache database.
- [memoize](https://grantjenks.com/docs/diskcache/api.html#diskcache.Cache.memoize)(name=None, typed=False, expire=None, tag=None, ignore=()): Memoizing cache decorator.
- [read](https://grantjenks.com/docs/diskcache/api.html#diskcache.Cache.read)(key, retry=False): Return file handle value corresponding to key from cache.
- [incr](https://grantjenks.com/docs/diskcache/api.html#diskcache.Cache.incr)(key, delta=1, default=0, retry=False): Increment value by delta for item with key.
- [decr](https://grantjenks.com/docs/diskcache/api.html#diskcache.Cache.decr)(key, delta=1, default=0, retry=False): Decrement value by delta for item with key.
- [iterkeys](https://grantjenks.com/docs/diskcache/api.html#diskcache.Cache.iterkeys)(reverse=False): Iterate Cache keys in database sort order.
- [peek](https://grantjenks.com/docs/diskcache/api.html#diskcache.Cache.peek)(prefix=None, default=(None, None), side='front', expire_time=False, tag=False, retry=False): Peek at key and value item pair from side of queue in cache.
- [peekitem](https://grantjenks.com/docs/diskcache/api.html#diskcache.Cache.peekitem)(last=True, expire_time=False, tag=False, retry=False): Peek at key and value item pair in cache based on iteration order.
- [push](https://grantjenks.com/docs/diskcache/api.html#diskcache.Cache.push)(value, prefix=None, side='back', expire=None, read=False, tag=None, retry=False): Push value onto side of queue identified by prefix in cache.
- [pull](https://grantjenks.com/docs/diskcache/api.html#diskcache.Cache.pull)(prefix=None, default=(None, None), side='front', expire_time=False, tag=False, retry=False): Pull key and value item pair from side of queue in cache.
- [transact](https://grantjenks.com/docs/diskcache/api.html#diskcache.Cache.transact)(retry=False): Context manager to perform a transaction by locking the cache.
- [stats](https://grantjenks.com/docs/diskcache/api.html#diskcache.Cache.stats)(enable=True, reset=False): Return cache statistics hits and misses.
- [volume](https://grantjenks.com/docs/diskcache/api.html#diskcache.Cache.volume)(): Return estimated total size of cache on disk.
- [check](https://grantjenks.com/docs/diskcache/api.html#diskcache.Cache.check)(fix=False, retry=False)¶: Check database and file system consistency. Intended for use in testing and post-mortem error analysis.
- [reset](https://grantjenks.com/docs/diskcache/api.html#diskcache.Cache.reset)(key, value=ENOVAL, update=True): Reset key and value item from Settings table.
- [cull](https://grantjenks.com/docs/diskcache/api.html#diskcache.Cache.cull)(retry=False): Cull items from cache until volume is less than size limit.

## Basic

In [26]:
cache.set("my-key", "my-value")
cache.get("my-key")

'my-value'

In [27]:
"my-key" in cache

True

In [28]:
print(cache.get("unknown-key"))

None


In [29]:
"unknown-key" in cache

False

## Cache with expire

In [30]:
key = "my-key"
cache.set(key, "my-value", expire=1)
print(f"first time: {cache.get(key)}")
time.sleep(1.5)
print(f"second time: {cache.get(key)}")

first time: my-value
second time: None


## Cache with Tag

Tag is the metadata of a cached item. Tag value can be int, float, string, bytes or None. Tag can be used to

In [31]:
key = "my-key"
cache.set(key, "my-value", tag="my-tag")
cache.get(key)

'my-value'

In [33]:
cache.evict("my-tag")
print(cache.get(key))

None


## Memoize

In [36]:
class CloudData:
    def __init__(self, backend_id: str):
        self.backend_id = backend_id

    def read_data(self):
        print("reading data from cloud")
        return [
            {"name": "alice"},
            {"name": "bob"},
            {"name": "cathy"},
        ]

    @cache.memoize(expire=1)
    def query_data(self, name):
        data = self.read_data()
        return [d for d in data if d["name"] == name]

cloud_data_1 = CloudData("backend-1")

print("query 1")
print(cloud_data_1.query_data(name="alice"))

print("query 2")
print(cloud_data_1.query_data(name="alice"))

time.sleep(1.5)
print("query 3")
print(cloud_data_1.query_data(name="alice"))

query 1
reading data from cloud
[{'name': 'alice'}]
query 2
[{'name': 'alice'}]
query 3
reading data from cloud
[{'name': 'alice'}]
